This is my python file for my model

In [115]:
# Mounting my Google Drive
# comment out if using on VsCode
from google.colab import drive
import os

mountPath = '/content/drive'
drive.mount(mountPath, force_remount=True)#Forcing remount if needed

Mounted at /content/drive


# Chapter 1 - downloading and altering the first Dataset

In [116]:
#cloning full github rather than just folders, this means everything should be up to date in content
%cd /content
!rm -rf DementiaDetection
!git clone https://github.com/Unbreakable60000/DementiaDetection.git
%cd DementiaDetection

/content
Cloning into 'DementiaDetection'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 125 (delta 57), reused 47 (delta 17), pack-reused 0 (from 0)
Receiving objects: 100% (125/125), 7.89 MiB | 13.15 MiB/s, done.
Resolving deltas: 100% (57/57), done.
/content/DementiaDetection


Dataset before alteration can be accessed here:
https://zenodo.org/records/7622128

In [117]:
import pandas as pd

activityDF = pd.read_csv("/content/drive/MyDrive/synoptic/oldDataset/Activity.csv")
demographicsDF = pd.read_csv("/content/drive/MyDrive/synoptic/oldDataset/Demographics.csv")
labelsDf = pd.read_csv("/content/drive/MyDrive/synoptic/oldDataset/Labels.csv")
physiologyDf = pd.read_csv("/content/drive/MyDrive/synoptic/oldDataset/Physiology.csv")
sleepDf = pd.read_csv("/content/drive/MyDrive/synoptic/oldDataset/Sleep.csv")

dfItems = {"activityDf": activityDF,"demographicsDF": demographicsDF,"labelsDf": labelsDf,"physiologyDf": physiologyDf, "sleepDf":sleepDf}

In [118]:
for name,df in dfItems.items():#itterates through list and gives column names and how many unique values are in each
    print(f"{name}:")
    print("--------")
    print(df.columns.tolist())
    for col in df.columns:
        num_unique = df[col].nunique()
        print(f"{col}: {num_unique} unique values")
    print()

activityDf:
--------
['patient_id', 'location_name', 'date']
patient_id: 56 unique values
location_name: 8 unique values
date: 916821 unique values

demographicsDF:
--------
['patient_id', 'age', 'sex']
patient_id: 56 unique values
age: 3 unique values
sex: 2 unique values

labelsDf:
--------
['patient_id', 'date', 'type']
patient_id: 49 unique values
date: 518 unique values
type: 6 unique values

physiologyDf:
--------
['patient_id', 'date', 'device_type', 'value', 'unit']
patient_id: 55 unique values
date: 9773 unique values
device_type: 8 unique values
value: 2686 unique values
unit: 5 unique values

sleepDf:
--------
['patient_id', 'date', 'state', 'heart_rate', 'respiratory_rate', 'snoring']
patient_id: 17 unique values
date: 92180 unique values
state: 4 unique values
heart_rate: 68 unique values
respiratory_rate: 24 unique values
snoring: 2 unique values



The above shows 56 unique patients, I will now join all into one csv document. And then try and find more data to include. The current issue is that there are multiple for some individuals. First I am not including activity as it provided information about the indiduduals movements and thats not an area I am focusing on

In [119]:
print("dfItem removed:")
dfItems.pop("activityDf")

dfItem removed:


,patient_id,location_name,date
0,0697d,Fridge Door,2019-06-28 13:03:29
1,0697d,Kitchen,2019-06-28 13:11:44
2,0697d,Front Door,2019-06-28 13:13:50
3,0697d,Bedroom,2019-06-28 13:13:53
4,0697d,Fridge Door,2019-06-28 13:14:09
...,...,...,...
1030554,fd100,Hallway,2019-06-30 23:48:50
1030555,fd100,Lounge,2019-06-30 23:49:40
1030556,fd100,Kitchen,2019-06-30 23:50:02
1030557,fd100,Front Door,2019-06-30 23:51:28


Now Activity is removed I can continue cleaning

In [120]:
#dropping dates as they arent needed
labelsDf = labelsDf.drop(columns=['date'])
physiologyDf = physiologyDf.drop(columns=['date'])
sleepDf = sleepDf.drop(columns=['date'])

In [121]:
labelsDf.head()

,patient_id,type
0,c55f8,Blood pressure
1,16f4b,Blood pressure
2,16f4b,Agitation
3,ec812,Blood pressure
4,16f4b,Agitation


In [122]:
#changing labelsdf to have multiple columns with 0 or 1 for each of the type conditions (this will make it easier to merge later)

labelsDf = (
    labelsDf
    .pivot_table(
        index='patient_id',
        columns='type',
        aggfunc='size',
        fill_value=0
    )
    .reset_index()
)
# Converting counts to presence (0/1) - this will be better for my models later on
label_cols = labelsDf.columns.drop('patient_id')
labelsDf[label_cols] = (labelsDf[label_cols] > 0).astype(int)

In [123]:
labelsDf.head()

type,patient_id,Agitation,Blood pressure,Body temperature,Body water,Pulse,Weight
0,0697d,0,1,0,0,1,0
1,099bc,1,1,0,0,0,0
2,0cda9,1,0,0,1,0,0
3,0d5ef,1,1,0,0,1,0
4,0efe8,0,0,0,1,1,0


Next I have expanded physiology by moving it to long format and grouping by patient_id - naming conventions and labelling will be done after merging

In [124]:
physiologyDf = (
    physiologyDf
    .pivot_table(
        index='patient_id',
        columns='device_type',
        values='value',
        aggfunc='mean'   # average per patient per device
    )
    .reset_index()
)

In [125]:
physiologyDf.head()

device_type,patient_id,Body Temperature,Body weight,Diastolic blood pressure,Heart rate,O/E - muscle mass,Skin Temperature,Systolic blood pressure,Total body water
0,0697d,36.386688,86.200000,80.000000,53.400000,64.550000,NaN,156.600000,50.900000
1,099bc,36.769330,52.614000,87.253333,69.226667,37.178378,NaN,151.466667,51.532432
2,0d5ef,36.572138,99.614286,88.266667,80.377778,67.933333,NaN,145.044444,48.633333
3,0efe8,36.370707,70.353731,75.800000,72.733333,39.607895,NaN,135.288889,42.381579
4,0f352,36.286647,72.400000,78.222222,74.000000,NaN,NaN,137.222222,NaN


In [126]:
sleepDf.head()

,patient_id,state,heart_rate,respiratory_rate,snoring
0,0f352,AWAKE,69.0,14.0,False
1,0f352,AWAKE,66.0,14.0,False
2,0f352,AWAKE,70.0,14.0,False
3,0f352,AWAKE,70.0,13.0,False
4,0f352,AWAKE,68.0,13.0,False


In [127]:
sleepDf = (
    sleepDf
    .pivot_table(
        index='patient_id',
        columns='state',
        aggfunc='size',
        fill_value=0
    )
    .reset_index()
)

# Converting counts to presence (0/1) - this will be better for my models later on (binary)
state_cols = sleepDf.columns.drop('patient_id')
sleepDf[state_cols] = (sleepDf[state_cols] > 0).astype(int)

In [128]:
sleepDf.head()

state,patient_id,AWAKE,DEEP,LIGHT,REM
0,0f352,1,1,1,1
1,16f4b,1,1,1,1
2,1fbe4,1,1,1,1
3,30a32,1,1,1,1
4,55cd4,1,1,1,1


In [129]:
#now all have been aggregated I can merge them

In [130]:
from functools import reduce #merging the new aggregated files with the demographics "patients" df

dfs = [demographicsDF, labelsDf, physiologyDf, sleepDf]

mergedDf = reduce(
    lambda left, right: pd.merge(left, right, on='patient_id', how='left'),
    dfs
)

In [131]:
mergedDf.head()

,patient_id,age,sex,Agitation,Blood pressure,Body temperature,Body water,Pulse,Weight,Body Temperature,...,Diastolic blood pressure,Heart rate,O/E - muscle mass,Skin Temperature,Systolic blood pressure,Total body water,AWAKE,DEEP,LIGHT,REM
0,b9d58,"(70, 80]",Female,1.0,0.0,0.0,0.0,0.0,0.0,36.330500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c55f8,"(80, 90]",Female,0.0,1.0,0.0,0.0,1.0,1.0,36.728481,...,82.730000,60.450000,35.636585,34.529511,144.98,47.571951,1.0,1.0,1.0,1.0
2,16f4b,"(80, 90]",Male,1.0,1.0,0.0,0.0,0.0,0.0,36.745277,...,88.785714,78.785714,NaN,34.586048,146.00,NaN,1.0,1.0,1.0,1.0
3,fd100,"(90, 110]",Female,0.0,1.0,0.0,0.0,0.0,0.0,36.627000,...,75.333333,56.333333,37.500000,NaN,148.00,48.700000,NaN,NaN,NaN,NaN
4,1fbe4,"(80, 90]",Male,0.0,1.0,0.0,0.0,1.0,0.0,36.034663,...,78.876923,54.215385,60.618919,33.088047,146.00,51.121622,1.0,1.0,1.0,1.0


In [132]:
outputPath = "/content/DementiaDetection/datasetFolder/mergedDataset.csv"
mergedDf.to_csv(outputPath, index=False)

# Chapter 2 - Looking into creating a synthetic dataset

## Why choose this?

As of limitations with gaining access to data I've decided to look into generating my own dataset with what I need based on field for questions and statistics produced by reputable sites like the NHS or .Gov

## Fields I will have:

1. **patientID** - random hash not repeatable

2. **Age** - 30-100 (decide later exact ages)

3. **Gender** - Male (0) or Female (1) (using binary where possible)

4. **dementiaHistory** - No(0) or Yes(1) - does the patient have a history of dementia in the family
5. **cognitionMedications** - No(0) or Yes(1) - is the patient on medications that could affect cognition
6. **cardiometabolic** - No(0) or Yes(1) - does the patient have conditions like hypertension, diabetes, stroke, or heart disease?
7. **Smoked** - No(0) or Yes(1) - has the patient ever smoked?
8. **Alchohol** - low(0) or moderate(1) or high(2) [ordinal datawise] - average alchohol consumption level
9. **physical** - low(0) or moderate(1) or high(2) [ordinal datawise] - average physical activity level
10. **sleep** - poor(0) or average(1) or good(2) [ordinal datawise] - average sleep stats for patient
11.** memory** - No (0) or Yes(1) - does the patient often forget things?
12. **mentalState** - No (0) or Yes(1) - does the patient experience any signs of depression or anxiety
13. **diet** - No (0) or Yes(1) - does the patient have a high carb diet
14. **bloodPressure** - No (0) or Yes(1) - does the patient have a high blood pressure
15. **Hearing** - No (0) or Yes(1) - does the patient have hearing loss
16. **Dementia** - percentage wise

The final dementia yes or no will be generated based on the statistics for the other columns using some form of linear model weighted on each. The probabilities are needed to be found and multiplied against each over for this generation.











## Researching the fields:

Percentage chances:

### Age

AlzheimersSociety (2022) and Office for Health Improvement & Disparities (2025)provided the AGE stats

**AGE:**
*WITH DEMENTIA*
1/14 overall population aged > 65
1/6 overall populationaged > 80
1/20 of people with dementia < 65 - important to understand this is with dementia

70800 people in the UK 2022 had early onset dementia out of 67,596,000 people in the UK general population in mid-2022. Population from the Office for National Statistics(2024).

$$
\frac{70800}{67,596,000} \times 100 = 0.104739925\% > 65 (2022)
$$
1/14 > 65 using AlzheimersSociety in 2022 = 7.14% of general population at that age

1/6 > 80 using AlzheimersSociety in 2022 = 16.67% general population at that age

Assuming > 65 < 80 = 7.14% as it was stated by AlzheimersSociety

AGE bracket | Percentage of having Dementia - these

| Age group | Percentage of having Dementia Approximately |
|-----------|-------------------|
| <65       | ~0.104% |
| >=65<80   | ~7.14%  |
| >=80      | ~16.67% |

FIND 60 PLUS 70 PLUS AND SO ON DO THIS!!!!!

# Chapter 3 - looking into more Datasets

### Whats the issue with synthetic data in training?
Synthetic data is not real and sometimes generated this means it can't be relied on for training. It seems like a good approach but data is always more informative if it is real. Therefore I have itterated and changed my approach:


Focus on alzheimer's as there is more data on this element of dementia
ADNI - most famous study
Focus on alzheimer's

Applied to
https://adni.loni.usc.edu/data-samples/adni-data/

https://adni.loni.usc.edu/help-faqs/adni-documentation/

### ADNI:

Part 1 could focus on Alzheimer's as it is easier to detect and there is more data on it (modelling)

Part 2 could focus on "if not Alzheimers what other dementia could it be?"

^this wont be based on a model this will just be a function based of some questions and gives a percentage back for each.

I have applied to ADNI as Senior Lecturer Dr Indranath Chatterjee advised it:

"The Alzheimer's Disease Neuroimaging Initiative (ADNI) is a longitudinal, multi-center, observational study.
The overall goal of ADNI is to validate biomarkers for Alzheimer's disease (AD) clinical trials." (Alzheimer's Disease Neuroimaging Initiative, 2026)



# Chapter 4 - ADNI

After being accepted to ADNI I explored and downloaded to datasets:

1. PTDEMOG (Participant Demographics): Very useful. Contains age, sex, education, ethnicity, and other baseline participant info
2. DXSUM (Diagnosis Summary): Crucial. This gives the clinical diagnosis of each participant at different visits: cognitively normal (CN), mild cognitive impairment (MCI), Alzheimer's disease (AD)

I will upload them to gitHub and then clean and merge them to then start my data analysis then modelling process





In [133]:
# Same proccess as before that I did for the smaller data set
DXSUMDF = pd.read_csv("/content/DementiaDetection/datasetFolder/ADNIDataset/DXSUM.csv")
PTDEMOGDF = pd.read_csv("/content/DementiaDetection/datasetFolder/ADNIDataset/PTDEMOG.csv")
MEDHISTDF = pd.read_csv("/content/DementiaDetection/datasetFolder/ADNIDataset/MEDHIST.csv")
PHYSICALDF = pd.read_csv("/content/DementiaDetection/datasetFolder/ADNIDataset/PHYSICAL.csv")
VITALSDF = pd.read_csv("/content/DementiaDetection/datasetFolder/ADNIDataset/VITALS.csv")

In [134]:
dfADNIItems = {"DXSUM":DXSUMDF,"PTDEMOG":PTDEMOGDF,"MEDHIST":MEDHISTDF,"PHYSICAL":PHYSICALDF,"VITALS":VITALSDF}
for name,df in dfADNIItems.items():#itterates through list and gives column names and how many unique values are in each
    print(f"{name}:")
    print("--------")
    print(df.columns.tolist())
    for col in df.columns:
        num_unique = df[col].nunique()
        print(f"{col}: {num_unique} unique values")
    print()

DXSUM:
--------
['PHASE', 'PTID', 'RID', 'VISCODE', 'VISCODE2', 'EXAMDATE', 'DIAGNOSIS', 'DXNORM', 'DXNODEP', 'DXMCI', 'DXMDES', 'DXMPTR1', 'DXMPTR2', 'DXMPTR3', 'DXMPTR4', 'DXMPTR5', 'DXMPTR6', 'DXMDUE', 'DXMOTHET', 'DXDSEV', 'DXDDUE', 'DXAD', 'DXAPP', 'DXAPROB', 'DXAPOSS', 'DXPARK', 'DXPDES', 'DXPCOG', 'DXPATYP', 'DXDEP', 'DXOTHDEM', 'DXODES', 'DXCONFID', 'ID', 'SITEID', 'USERDATE', 'USERDATE2', 'DD_CRF_VERSION_LABEL', 'LANGUAGE_CODE', 'HAS_QC_ERROR', 'update_stamp']
PHASE: 5 unique values
PTID: 3788 unique values
RID: 3788 unique values
VISCODE: 31 unique values
VISCODE2: 43 unique values
EXAMDATE: 4375 unique values
DIAGNOSIS: 3 unique values
DXNORM: 2 unique values
DXNODEP: 2 unique values
DXMCI: 2 unique values
DXMDES: 5 unique values
DXMPTR1: 3 unique values
DXMPTR2: 3 unique values
DXMPTR3: 4 unique values
DXMPTR4: 4 unique values
DXMPTR5: 3 unique values
DXMPTR6: 3 unique values
DXMDUE: 3 unique values
DXMOTHET: 18 unique values
DXDSEV: 4 unique values
DXDDUE: 2 unique values


## DXSUM - filtering Columns

As you can see the datasets have a lot of values, this will take time to clean, LETS GET STARTED!

In [135]:
#cleaning starting with DXSUMDF
DXSUMDF.head()

,PHASE,PTID,RID,VISCODE,VISCODE2,EXAMDATE,DIAGNOSIS,DXNORM,DXNODEP,DXMCI,...,DXODES,DXCONFID,ID,SITEID,USERDATE,USERDATE2,DD_CRF_VERSION_LABEL,LANGUAGE_CODE,HAS_QC_ERROR,update_stamp
0,ADNI1,011_S_0002,2,bl,bl,2005-09-29,1.0,1.0,-4.0,-4.0,...,-4.0,4.0,2,107,2005-10-01,NaN,NaN,NaN,NaN,2005-10-01 00:00:00
1,ADNI1,011_S_0003,3,bl,bl,2005-09-30,3.0,-4.0,-4.0,-4.0,...,-4.0,3.0,4,107,2005-10-01,NaN,NaN,NaN,NaN,2005-10-01 00:00:00
2,ADNI1,011_S_0005,5,bl,bl,2005-09-30,1.0,1.0,-4.0,-4.0,...,-4.0,4.0,6,107,2005-10-01,NaN,NaN,NaN,NaN,2005-10-01 00:00:00
3,ADNI1,011_S_0008,8,bl,bl,2005-09-30,1.0,1.0,-4.0,-4.0,...,-4.0,3.0,8,107,2005-10-01,NaN,NaN,NaN,NaN,2005-10-01 00:00:00
4,ADNI1,022_S_0007,7,bl,bl,2005-10-06,3.0,-4.0,-4.0,-4.0,...,-4.0,4.0,10,10,2005-10-06,NaN,NaN,NaN,NaN,2005-10-06 00:00:00


In [136]:
print(DXSUMDF['LANGUAGE_CODE'].unique()) #only nan (nothing) or e for english so can be dropped
print(DXSUMDF['LANGUAGE_CODE'].value_counts())

[nan 'e']
LANGUAGE_CODE
e    2431
Name: count, dtype: int64


In [137]:
print(DXSUMDF['HAS_QC_ERROR'].unique()) #no varability only one real value and so can be dropped
print(DXSUMDF['HAS_QC_ERROR'].value_counts())

[nan  0.]
HAS_QC_ERROR
0.0    2431
Name: count, dtype: int64


- update_stamp can also be dropped as it is just for recording when the database was updated - not useful for modelling
- DD_CRF_VERSION_LABEL has no varability and so is not useful for our model
- USERDATE2 is not clinical and tells us nothing about our patient so can be dropped
- same with USERDATE ^
- SITEID doesn't help determine dementia it just shows where ADNI scores for patient where recorded
- ID - is a unique identifier for rows not patients and could make our model learn the wrong idea - so we are dropping it

In [138]:
print(DXSUMDF['DXCONFID'].unique()) #stands for diagnosis confidence
print(DXSUMDF['DXCONFID'].value_counts())
#4 being very high, 3 being high, 2 being moderate and 1 being low confidence
#useful for our model so keeping it (can be removed later if not needed)

[ 4.  3.  1.  2. nan]
DXCONFID
4.0    2369
3.0    1244
2.0     222
1.0      33
Name: count, dtype: int64


In [139]:
#dropping columns so far...
DXSUMDFV2 = DXSUMDF.drop(["LANGUAGE_CODE","HAS_QC_ERROR","update_stamp","DD_CRF_VERSION_LABEL","USERDATE2","USERDATE",
 "SITEID","ID" ], axis=1)

In [140]:
DXSUMDFV2.head()

,PHASE,PTID,RID,VISCODE,VISCODE2,EXAMDATE,DIAGNOSIS,DXNORM,DXNODEP,DXMCI,...,DXAPROB,DXAPOSS,DXPARK,DXPDES,DXPCOG,DXPATYP,DXDEP,DXOTHDEM,DXODES,DXCONFID
0,ADNI1,011_S_0002,2,bl,bl,2005-09-29,1.0,1.0,-4.0,-4.0,...,-4,-4,-4.0,-4.0,-4.0,-4.0,NaN,-4.0,-4.0,4.0
1,ADNI1,011_S_0003,3,bl,bl,2005-09-30,3.0,-4.0,-4.0,-4.0,...,1,-4,-4.0,-4.0,-4.0,-4.0,NaN,-4.0,-4.0,3.0
2,ADNI1,011_S_0005,5,bl,bl,2005-09-30,1.0,1.0,-4.0,-4.0,...,-4,-4,-4.0,-4.0,-4.0,-4.0,NaN,-4.0,-4.0,4.0
3,ADNI1,011_S_0008,8,bl,bl,2005-09-30,1.0,1.0,-4.0,-4.0,...,-4,-4,-4.0,-4.0,-4.0,-4.0,NaN,-4.0,-4.0,3.0
4,ADNI1,022_S_0007,7,bl,bl,2005-10-06,3.0,-4.0,-4.0,-4.0,...,3,-4,-4.0,-4.0,-4.0,-4.0,NaN,-4.0,-4.0,4.0


- keeping Phase for now, but probably will drop later, just shows which ADNI phase each patient got recorded
- PTID - the patient ID - the unique identifier - this is the linker between
- RID = identifies the specific visit for that patient - so useful to have not dropping currently
- Viscodes and examdates - i want to explore more so not dropping yet

In [141]:
print(DXSUMDFV2['DIAGNOSIS'].unique()) #stands for diagnosis (VERY USEFUL NOT DROPPING)
print(DXSUMDFV2['DIAGNOSIS'].value_counts())

[ 1.  3.  2. nan]
DIAGNOSIS
2.0    6610
1.0    6369
3.0    3008
Name: count, dtype: int64



1.0 = 	CN (Cognitively Normal) | 2.0	= MCI (Mild Cognitive Impairment) | 3.0	= AD (Alzheimer's Disease) | nan	= Missing / not assessed

In [142]:
print(DXSUMDFV2['DXNORM'].unique())
print(DXSUMDFV2['DXNORM'].value_counts())
#might drop later as it could overcomplicate processing and has a lot of non entered
#details
# basically represents cognitively normal (1) vs non or not assessed

[ 1. -4. nan]
DXNORM
-4.0    2738
 1.0    1130
Name: count, dtype: int64


In [143]:
print(DXSUMDFV2['DXNODEP'].unique()) #this is a depression related column, it would be useful
#but its mainly missing so dropping it is probably a good choice (unless I want to add it later)
print(DXSUMDFV2['DXNODEP'].value_counts())

[-4.  1. nan]
DXNODEP
-4.0    3783
 1.0      85
Name: count, dtype: int64


In [144]:
print(DXSUMDFV2['DXMCI'].unique()) #Mild cognitive impairment
print(DXSUMDFV2['DXMCI'].value_counts())

[-4.  1. nan]
DXMCI
-4.0    2267
 1.0    1601
Name: count, dtype: int64


1 meaning the patient does and -4 meaning not assessed, can be derived from Diagnosis column so dropping it

In [145]:
print(DXSUMDFV2['DXMDES'].unique()) #Mild cognitive impairment subtype
print(DXSUMDFV2['DXMDES'].value_counts())

['-4' '1' '1|02' '2' '1|2' nan]
DXMDES
1       5983
-4      5482
1|2      296
1|02     167
2        124
Name: count, dtype: int64


In [146]:
print(DXSUMDFV2['DXODES'].unique()) #other disorders column very sparse dropping it
print(DXSUMDFV2['DXODES'].value_counts())

[-4. 14.  1. nan  4. 12.  6.  9.  7.]
DXODES
-4.0     3860
 14.0      29
 1.0       12
 12.0       6
 9.0        6
 4.0        3
 6.0        2
 7.0        1
Name: count, dtype: int64


In [149]:
DXSUMDFV3 = DXSUMDFV2.drop(["DXNODEP","DXMCI","DXMDES","DXODES"], axis=1)
DXSUMDFV3.head()

,PHASE,PTID,RID,VISCODE,VISCODE2,EXAMDATE,DIAGNOSIS,DXNORM,DXMPTR1,DXMPTR2,...,DXAPP,DXAPROB,DXAPOSS,DXPARK,DXPDES,DXPCOG,DXPATYP,DXDEP,DXOTHDEM,DXCONFID
0,ADNI1,011_S_0002,2,bl,bl,2005-09-29,1.0,1.0,-4.0,-4.0,...,-4.0,-4,-4,-4.0,-4.0,-4.0,-4.0,NaN,-4.0,4.0
1,ADNI1,011_S_0003,3,bl,bl,2005-09-30,3.0,-4.0,-4.0,-4.0,...,1.0,1,-4,-4.0,-4.0,-4.0,-4.0,NaN,-4.0,3.0
2,ADNI1,011_S_0005,5,bl,bl,2005-09-30,1.0,1.0,-4.0,-4.0,...,-4.0,-4,-4,-4.0,-4.0,-4.0,-4.0,NaN,-4.0,4.0
3,ADNI1,011_S_0008,8,bl,bl,2005-09-30,1.0,1.0,-4.0,-4.0,...,-4.0,-4,-4,-4.0,-4.0,-4.0,-4.0,NaN,-4.0,3.0
4,ADNI1,022_S_0007,7,bl,bl,2005-10-06,3.0,-4.0,-4.0,-4.0,...,1.0,3,-4,-4.0,-4.0,-4.0,-4.0,NaN,-4.0,4.0


subtypes overcomplicates the model so dropping it

- Dropping Viscode and Viscode 2 as ExamDate will show when they got their diagnosis (diagnosis date column basically)

In [150]:
print(DXSUMDFV3['DXOTHDEM'].unique()) #has other dementia or not (dropping as very sparce)
print(DXSUMDFV3['DXOTHDEM'].value_counts())

[-4.  1. nan]
DXOTHDEM
-4.0    3860
 1.0       8
Name: count, dtype: int64


In [151]:
print(DXSUMDFV3['DXDEP'].unique()) #depression flag - could be useful for predicting - keeping it
print(DXSUMDFV3['DXDEP'].value_counts())

[nan  0.  1.]
DXDEP
0.0    10483
1.0     1045
Name: count, dtype: int64


In [152]:
print(DXSUMDFV3['DXPATYP'].unique()) #pathology type or subtype related to diagnosis (dropping as very sparce)
print(DXSUMDFV3['DXPATYP'].value_counts())

[-4.  7.  5. nan]
DXPATYP
-4.0    3866
 7.0       1
 5.0       1
Name: count, dtype: int64


In [153]:
print(DXSUMDFV3['DXPCOG'].unique()) #cognitive description very sparce so dropping it
print(DXSUMDFV3['DXPCOG'].value_counts())

[-4.  4.  3. nan]
DXPCOG
-4.0    3864
 3.0       3
 4.0       1
Name: count, dtype: int64


In [154]:
print(DXSUMDFV3['DXPDES'].unique()) #Parkinson's disease -very sparce so dropping it
print(DXSUMDFV3['DXPDES'].value_counts())

[-4.  3.  4. nan]
DXPDES
-4.0    3862
 3.0       4
 4.0       2
Name: count, dtype: int64


In [155]:
print(DXSUMDFV3['DXPARK'].unique()) #Parkinson's disease 0 or 1, could help with prediction so keeping it
print(DXSUMDFV3['DXPARK'].value_counts())

[-4.  1.  0. nan]
DXPARK
 0.0    11839
-4.0     3862
 1.0      162
Name: count, dtype: int64


In [156]:
print(DXSUMDFV3['DXAPOSS'].unique()) #huge amount of missing data so dropping it - also can be done with diagnosis column
#so it isn't needed
print(DXSUMDFV3['DXAPOSS'].value_counts())

['-4' '3' '1|03|05' '7' '3|06' '1' '2' '6' '1|03' '5' '1|5' '5|7' '3|7'
 '1|2' nan '1|7']
DXAPOSS
-4         9946
1            34
7            32
2            14
3             8
3|06          3
5             3
1|5           2
6             2
1|7           2
1|03|05       1
1|03          1
5|7           1
3|7           1
1|2           1
Name: count, dtype: int64


In [157]:
print(DXSUMDFV3['DXAPROB'].unique()) #hard for model to learn as of multiple codes and sparceness so dropping
print(DXSUMDFV3['DXAPROB'].value_counts())

['-4' '1' '3' '2' '5' '1|03' '7' '2|07' '6' '3|05' '5|07' nan]
DXAPROB
-4      2771
1        990
3         70
2         15
7         11
5          4
3|05       3
1|03       1
2|07       1
6          1
5|07       1
Name: count, dtype: int64


In [158]:
print(DXSUMDFV3['DXAPP'].unique()) #redundant with diagnosis column so dropping it
print(DXSUMDFV3['DXAPP'].value_counts())

[-4.  1.  2. nan]
DXAPP
 1.0    2783
-4.0    2739
 2.0     106
Name: count, dtype: int64


In [160]:
DXSUMDFV4 = DXSUMDFV3.drop(["VISCODE","VISCODE2","DXOTHDEM","DXPATYP","DXPCOG","DXPDES","DXAPOSS","DXAPROB","DXAPP"], axis=1)
DXSUMDFV4.head()

,PHASE,PTID,RID,EXAMDATE,DIAGNOSIS,DXNORM,DXMPTR1,DXMPTR2,DXMPTR3,DXMPTR4,DXMPTR5,DXMPTR6,DXMDUE,DXMOTHET,DXDSEV,DXDDUE,DXAD,DXPARK,DXDEP,DXCONFID
0,ADNI1,011_S_0002,2,2005-09-29,1.0,1.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4,-4.0,NaN,-4.0,-4.0,NaN,4.0
1,ADNI1,011_S_0003,3,2005-09-30,3.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4,2.0,NaN,1.0,-4.0,NaN,3.0
2,ADNI1,011_S_0005,5,2005-09-30,1.0,1.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4,-4.0,NaN,-4.0,-4.0,NaN,4.0
3,ADNI1,011_S_0008,8,2005-09-30,1.0,1.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4,-4.0,NaN,-4.0,-4.0,NaN,3.0
4,ADNI1,022_S_0007,7,2005-10-06,3.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4,1.0,NaN,1.0,-4.0,NaN,4.0


Decided to drop phase as it isnt needed for evaluation and might confuse the model (treating all phases as one)

In [161]:
print(DXSUMDFV3['DXNORM'].unique()) #removing as can be done with diagnosis adds no predictive power
print(DXSUMDFV3['DXNORM'].value_counts())

[ 1. -4. nan]
DXNORM
-4.0    2738
 1.0    1130
Name: count, dtype: int64


In [162]:
print(DXSUMDFV3['DXMPTR1'].unique()) #For single-visit prediction, it likely won’t improve the model
#will remove all of DXMPTR
print(DXSUMDFV3['DXMPTR1'].value_counts())

[-4.  1.  0. nan]
DXMPTR1
 1.0    6333
-4.0    2276
 0.0     190
Name: count, dtype: int64


In [164]:
print(DXSUMDFV3['DXMDUE'].unique()) #redundant with diagnosis so dropping
print(DXSUMDFV3['DXMDUE'].value_counts())

[-4.  1.  2. nan]
DXMDUE
 1.0    6085
-4.0    2267
 2.0     483
Name: count, dtype: int64


In [165]:
DXSUMDFV5 = DXSUMDFV4.drop(["PHASE","DXNORM","DXMPTR1","DXMPTR2","DXMPTR3","DXMPTR4","DXMPTR5","DXMPTR6","DXMDUE"], axis=1)
DXSUMDFV5.head()

,PTID,RID,EXAMDATE,DIAGNOSIS,DXMOTHET,DXDSEV,DXDDUE,DXAD,DXPARK,DXDEP,DXCONFID
0,011_S_0002,2,2005-09-29,1.0,-4,-4.0,NaN,-4.0,-4.0,NaN,4.0
1,011_S_0003,3,2005-09-30,3.0,-4,2.0,NaN,1.0,-4.0,NaN,3.0
2,011_S_0005,5,2005-09-30,1.0,-4,-4.0,NaN,-4.0,-4.0,NaN,4.0
3,011_S_0008,8,2005-09-30,1.0,-4,-4.0,NaN,-4.0,-4.0,NaN,3.0
4,022_S_0007,7,2005-10-06,3.0,-4,1.0,NaN,1.0,-4.0,NaN,4.0


In [167]:
print(DXSUMDFV5['DXMOTHET'].unique()) #sparse and over complex with rare cases so dropping
print(DXSUMDFV5['DXMOTHET'].value_counts())

['-4' '9' '14' '1' '9|14' '7|9' '7' '4' '2' '2|14' nan '5' '12' '5|7|14'
 '7|14' '11' '6' '1|12' '5|14']
DXMOTHET
-4        9790
14         267
9          119
7           31
1           13
9|14        10
12           7
2|14         6
7|14         6
2            3
5            3
5|7|14       3
4            2
7|9          1
11           1
6            1
1|12         1
5|14         1
Name: count, dtype: int64


In [168]:
print(DXSUMDFV5['DXDDUE'].unique()) #sparse so removing
print(DXSUMDFV5['DXDDUE'].value_counts())

[nan  1.  2.]
DXDDUE
1.0    1813
2.0      51
Name: count, dtype: int64


In [169]:
print(DXSUMDFV5['DXAD'].unique()) #Alzheimer’s disease flag redudant with diagnosis so dropping
print(DXSUMDFV5['DXAD'].value_counts())

[-4.  1. nan]
DXAD
-4.0    2739
 1.0    1129
Name: count, dtype: int64


In [170]:
DXSUMDFV6 =DXSUMDFV5.drop(["DXMOTHET","DXDDUE","DXAD"], axis=1)
DXSUMDFV6.head()

,PTID,RID,EXAMDATE,DIAGNOSIS,DXDSEV,DXPARK,DXDEP,DXCONFID
0,011_S_0002,2,2005-09-29,1.0,-4.0,-4.0,NaN,4.0
1,011_S_0003,3,2005-09-30,3.0,2.0,-4.0,NaN,3.0
2,011_S_0005,5,2005-09-30,1.0,-4.0,-4.0,NaN,4.0
3,011_S_0008,8,2005-09-30,1.0,-4.0,-4.0,NaN,3.0
4,022_S_0007,7,2005-10-06,3.0,1.0,-4.0,NaN,4.0


At version 6 I have filtered and analysed each feature and came out with the best ones from DXSUM. I've kept PTID and RID for merging later. Now I will move onto cleaning the next dataset

## MEDHIST - filtering Columns

In [171]:
MEDHISTDF.head()

,PHASE,PTID,RID,VISCODE,VISCODE2,VISDATE,MHSOURCE,MHPSYCH,MH2NEURL,MH3HEAD,...,MH16BSMOK,MH16CSMOK,MH17MALI,MH18SURG,MH19OTHR,ID,SITEID,USERDATE,USERDATE2,update_stamp
0,ADNI1,011_S_0002,2,sc,sc,2005-08-17,1,0,0,1,...,NaN,NaN,0,0,0,8,107,2005-08-17,NaN,2005-08-17 00:00:00
1,ADNI1,022_S_0001,1,f,f,2005-08-18,1,0,0,1,...,NaN,NaN,0,0,1,10,10,2005-08-18,NaN,2005-08-18 00:00:00
2,ADNI1,011_S_0003,3,sc,sc,2005-08-18,1,0,0,1,...,NaN,NaN,1,1,0,12,107,2005-08-18,NaN,2005-08-18 00:00:00
3,ADNI1,022_S_0004,4,sc,sc,2005-08-18,1,0,0,0,...,NaN,NaN,0,1,0,14,10,2005-08-18,NaN,2005-08-18 00:00:00
4,ADNI1,011_S_0005,5,sc,sc,2005-08-23,1,0,0,1,...,NaN,NaN,0,1,0,16,107,2005-08-23,NaN,2005-08-23 00:00:00


- Dropping Phase from all DF from now on as it is not needed for our modelling
- dropping VISCODE as its not predictive
- keeping visdate - age at visit
- keeping MHPSYCH as good for Psychiatric history prediction
- keeping MH2NEURL as good for neurological history prediction

In [173]:
print(MEDHISTDF['MH3HEAD'].unique()) #head injouries could be predictive so keeping it
print(MEDHISTDF['MH3HEAD'].value_counts())

[1 0]
MH3HEAD
1    1936
0    1147
Name: count, dtype: int64


In [174]:
print(MEDHISTDF['MH4CARD'].unique()) #cardiac history - could be predictive so keeping it
print(MEDHISTDF['MH4CARD'].value_counts())

[0 1]
MH4CARD
1    2048
0    1035
Name: count, dtype: int64


In [175]:
print(MEDHISTDF['MH16SMOK'].unique()) #binary smoking predictor (keeping)
print(MEDHISTDF['MH16SMOK'].value_counts())

[0 1]
MH16SMOK
0    1855
1    1228
Name: count, dtype: int64


In [176]:
print(MEDHISTDF['MH16ASMOK'].unique())#removing as too complex and sparse
print(MEDHISTDF['MH16ASMOK'].value_counts())

[ nan  1.  -4.   0.1  0.5  2.   3.5  1.5  3.   0.7  0.2  0.3  0.   6.
  2.5  4.  10.   5.   8.   0.8  1.2]
MH16ASMOK
-4.0     1142
 1.0      368
 0.5      133
 2.0       96
 1.5       45
 3.0       33
 0.0       26
 0.1       22
 2.5       13
 0.2       12
 0.3       10
 10.0       8
 5.0        5
 0.7        3
 0.8        3
 4.0        3
 1.2        2
 6.0        2
 3.5        1
 8.0        1
Name: count, dtype: int64


In [179]:
print(MEDHISTDF['MH16CSMOK'].unique()) #current smoking status (too sparse so dropping)
#removing all smoking except binary
print(MEDHISTDF['MH16CSMOK'].value_counts())

[ nan 45.  -4.   2.5 20.  19.  27.  40.   0.5 30.   5.   7.  47.  36.
 39.  15.   4.  33.  32.  21.  42.   9.   3.  38.  35.  50.  34.  23.
 17.  16.  49.  25.  59.  12.   1.  46.  28.   0.1 44.  37.   6.  52.
 24.  48.  43.  10.  14.  31.  41.  11.  29.  26.  56.  60.  18.  22.
 67.  51.  63.  54.  57.  13.  64.  55.  58.   8.   2.  53.  68.  12.5
  1.5]
MH16CSMOK
-4.0     1203
 40.0      54
 30.0      53
 50.0      34
 20.0      34
         ... 
 67.0       1
 64.0       1
 68.0       1
 12.5       1
 1.5        1
Name: count, Length: 70, dtype: int64


In [180]:
MEDHISTDFV2 =MEDHISTDF.drop(["PHASE","VISCODE","VISCODE2","MH16CSMOK","MH16ASMOK","MH16BSMOK",], axis=1)
MEDHISTDFV2.head()

,PTID,RID,VISDATE,MHSOURCE,MHPSYCH,MH2NEURL,MH3HEAD,MH4CARD,MH5RESP,MH6HEPAT,...,MH15BDRUG,MH16SMOK,MH17MALI,MH18SURG,MH19OTHR,ID,SITEID,USERDATE,USERDATE2,update_stamp
0,011_S_0002,2,2005-08-17,1,0,0,1,0,0,0,...,NaN,0,0,0,0,8,107,2005-08-17,NaN,2005-08-17 00:00:00
1,022_S_0001,1,2005-08-18,1,0,0,1,0,0,1,...,NaN,0,0,0,1,10,10,2005-08-18,NaN,2005-08-18 00:00:00
2,011_S_0003,3,2005-08-18,1,0,0,1,1,0,0,...,NaN,1,1,1,0,12,107,2005-08-18,NaN,2005-08-18 00:00:00
3,022_S_0004,4,2005-08-18,1,0,0,0,1,0,0,...,NaN,1,0,1,0,14,10,2005-08-18,NaN,2005-08-18 00:00:00
4,011_S_0005,5,2005-08-23,1,0,0,1,0,0,0,...,NaN,1,0,1,0,16,107,2005-08-23,NaN,2005-08-23 00:00:00


In [181]:
MEDHISTDFV3 =MEDHISTDFV2.drop(["ID","SITEID","USERDATE","USERDATE2","update_stamp"], axis=1)#dropping admin features other than merging features
MEDHISTDFV3.head()

,PTID,RID,VISDATE,MHSOURCE,MHPSYCH,MH2NEURL,MH3HEAD,MH4CARD,MH5RESP,MH6HEPAT,...,MH14AALCH,MH14BALCH,MH14CALCH,MH15DRUG,MH15ADRUG,MH15BDRUG,MH16SMOK,MH17MALI,MH18SURG,MH19OTHR
0,011_S_0002,2,2005-08-17,1,0,0,1,0,0,0,...,NaN,NaN,NaN,0,NaN,NaN,0,0,0,0
1,022_S_0001,1,2005-08-18,1,0,0,1,0,0,1,...,NaN,NaN,NaN,0,NaN,NaN,0,0,0,1
2,011_S_0003,3,2005-08-18,1,0,0,1,1,0,0,...,NaN,NaN,NaN,0,NaN,NaN,1,1,1,0
3,022_S_0004,4,2005-08-18,1,0,0,0,1,0,0,...,NaN,NaN,NaN,0,NaN,NaN,1,0,1,0
4,011_S_0005,5,2005-08-23,1,0,0,1,0,0,0,...,NaN,NaN,NaN,0,NaN,NaN,1,0,1,0


In [182]:
print(MEDHISTDFV3['MHSOURCE'].unique()) #dont need source so dropping
print(MEDHISTDFV3['MHSOURCE'].value_counts())

[1 2]
MHSOURCE
1    3062
2      21
Name: count, dtype: int64


In [183]:
print(MEDHISTDFV3['MH5RESP'].unique()) #respiritory history
print(MEDHISTDFV3['MH5RESP'].value_counts())

[0 1]
MH5RESP
0    2398
1     685
Name: count, dtype: int64


In [184]:
print(MEDHISTDFV3['MH6HEPAT'].unique()) #liver disease - mostly sparse so dropping
print(MEDHISTDFV3['MH6HEPAT'].value_counts())

[0 1]
MH6HEPAT
0    2963
1     120
Name: count, dtype: int64


In [185]:
print(MEDHISTDFV3['MH7DERM'].unique()) #unlikley to be predictive for dementia so might just add noise (so dropping)
print(MEDHISTDFV3['MH7DERM'].value_counts())
#same with MH8MUSCL muscle / skeletal so dropping that too

[0 1]
MH7DERM
0    2101
1     982
Name: count, dtype: int64


In [186]:
print(MEDHISTDFV3['MH9ENDO'].unique()) # Endocrine disorders can affect cognition, especially diabetes or thyroid problems
#so keeping it
print(MEDHISTDFV3['MH9ENDO'].value_counts())

[1 0]
MH9ENDO
0    1777
1    1306
Name: count, dtype: int64


In [187]:
print(MEDHISTDFV3['MH10GAST'].unique()) #gastrointestinal history (e.g., ulcers, liver issues, digestive disorders)
#may be low impact but can be deleted later if its not contributing to prediction
print(MEDHISTDFV3['MH10GAST'].value_counts())

[0 1]
MH10GAST
0    1710
1    1373
Name: count, dtype: int64


In [188]:
print(MEDHISTDFV3['MH11HEMA'].unique()) #hermatology - not common in my dataset so dropping it to reduce noise
print(MEDHISTDFV3['MH11HEMA'].value_counts())

[ 0  1 -4]
MH11HEMA
 0    2786
 1     296
-4       1
Name: count, dtype: int64


In [189]:
print(MEDHISTDFV3['MH12RENA'].unique()) #renal history (kidney disease, chronic kidney issues) keeping it
#may help prediction later
print(MEDHISTDFV3['MH12RENA'].value_counts())

[0 1]
MH12RENA
0    1745
1    1338
Name: count, dtype: int64


In [190]:
print(MEDHISTDFV3['MH13ALLE'].unique()) #allergies most likely have low predictive importance so dropping it
print(MEDHISTDFV3['MH13ALLE'].value_counts())

[0 1]
MH13ALLE
0    1790
1    1293
Name: count, dtype: int64


In [191]:
print(MEDHISTDFV3['MH14ALCH'].unique())#keeping but it is sparce, may help with prediction
print(MEDHISTDFV3['MH14ALCH'].value_counts())

[0 1]
MH14ALCH
0    2947
1     136
Name: count, dtype: int64


In [192]:
print(MEDHISTDFV3['MH14AALCH'].unique()) #dropping all alchol columns except MH14ALCH as they are sparce and overcomplex,
# this could overcomplicate my model
print(MEDHISTDFV3['MH14AALCH'].value_counts())

[nan  5.  4.  6.  3.  7.  8. 16.  2. 12.  9. 10. 24. 11. 25.  1.  0. 18.
 20. 15.]
MH14AALCH
5.0     17
4.0     15
3.0     13
8.0     10
2.0      9
6.0      7
9.0      4
16.0     2
7.0      2
10.0     2
18.0     2
0.0      2
20.0     2
12.0     1
24.0     1
1.0      1
25.0     1
11.0     1
15.0     1
Name: count, dtype: int64


In [193]:
print(MEDHISTDFV3['MH15DRUG'].unique()) #drug history is very sparce - dropping all drug columns for this reason
print(MEDHISTDFV3['MH15DRUG'].value_counts())

[0 1]
MH15DRUG
0    3053
1      30
Name: count, dtype: int64


In [195]:
MEDHISTDFV4 =MEDHISTDFV3.drop(["MHSOURCE","MH6HEPAT","MH7DERM","MH8MUSCL","MH11HEMA","MH14AALCH","MH14BALCH","MH14CALCH",
    "MH15DRUG","MH15ADRUG","MH15BDRUG"], axis=1)
MEDHISTDFV4.head()

,PTID,RID,VISDATE,MHPSYCH,MH2NEURL,MH3HEAD,MH4CARD,MH5RESP,MH9ENDO,MH10GAST,MH12RENA,MH13ALLE,MH14ALCH,MH16SMOK,MH17MALI,MH18SURG,MH19OTHR
0,011_S_0002,2,2005-08-17,0,0,1,0,0,1,0,0,0,0,0,0,0,0
1,022_S_0001,1,2005-08-18,0,0,1,0,0,0,0,0,1,0,0,0,0,1
2,011_S_0003,3,2005-08-18,0,0,1,1,0,1,0,1,0,0,1,1,1,0
3,022_S_0004,4,2005-08-18,0,0,0,1,0,0,0,1,0,0,1,0,1,0
4,011_S_0005,5,2005-08-23,0,0,1,0,0,0,1,1,0,0,1,0,1,0


In [196]:
print(MEDHISTDFV4['MH17MALI'].unique()) #history of malignancy (cancer, tumors, etc.)
#keeping it for now, might remove later
print(MEDHISTDFV4['MH17MALI'].value_counts())

[0 1]
MH17MALI
0    2281
1     802
Name: count, dtype: int64


In [197]:
print(MEDHISTDFV4['MH19OTHR'].unique()) #dropping as not specific enough
print(MEDHISTDFV4['MH19OTHR'].value_counts())

[0 1]
MH19OTHR
0    2424
1     659
Name: count, dtype: int64


In [198]:
print(MEDHISTDFV4['MH18SURG'].unique()) #dropping as not a strong predictor of dementia
print(MEDHISTDFV4['MH18SURG'].value_counts())

[0 1]
MH18SURG
1    2308
0     775
Name: count, dtype: int64


In [199]:
MEDHISTDFV5 =MEDHISTDFV4.drop(["MH19OTHR","MH18SURG"], axis=1)
MEDHISTDFV5.head()

,PTID,RID,VISDATE,MHPSYCH,MH2NEURL,MH3HEAD,MH4CARD,MH5RESP,MH9ENDO,MH10GAST,MH12RENA,MH13ALLE,MH14ALCH,MH16SMOK,MH17MALI
0,011_S_0002,2,2005-08-17,0,0,1,0,0,1,0,0,0,0,0,0
1,022_S_0001,1,2005-08-18,0,0,1,0,0,0,0,0,1,0,0,0
2,011_S_0003,3,2005-08-18,0,0,1,1,0,1,0,1,0,0,1,1
3,022_S_0004,4,2005-08-18,0,0,0,1,0,0,0,1,0,0,1,0
4,011_S_0005,5,2005-08-23,0,0,1,0,0,0,1,1,0,0,1,0


Filtered the columns, there are still a few that may be worse predictors but currently I cannot tell until I am modelling an anaysing once merged. I will now move onto the next DF which is PHYSICALDF

## PHYSICAL - filtering columns

# References

AlzheimersSociety (2022) What is dementia? Available at: https://www.alzheimers.org.uk/about-dementia/types-dementia/what-is-dementia(Accessed: 5 February 2026).

Office for Health Improvement & Disparities (2025) Dementia profile: prevalence and supporting well topics statistical commentary, March 2025. GOV.UK. Available at: https://www.gov.uk/government/statistics/dementia-profile-march-2025-update/dementia-profile-prevalence-and-supporting-well-topics-statistical-commentary-march-2025? (Accessed: 5 February 2026).

Office for National Statistics(2024) Population estimates for the UK, England, Wales, Scotland and Northern Ireland: mid-2022. Available at: https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/bulletins/annualmidyearpopulationestimates/mid2022 (Accessed: 5 February 2026).

Alzheimer's Disease Neuroimaging Initiative (ADNI) (2026) ADNI data. Available at: https://adni.loni.usc.edu/data-samples/adni-data/
(Accessed: 6 February 2026).